In [ ]:
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import RobustScaler, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import KFold, train_test_split
import matplotlib.pyplot as plt
import random
from torch_geometric import seed_everything
from torch_geometric.nn import GCNConv, GATConv, GraphConv, MFConv
from torch_geometric.nn import Linear
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import QM9
from torch_geometric.data import Batch
from tabulate import tabulate
import pandas as pd
import matplotlib.pyplot as plt
from permetrics.regression import RegressionMetric
from rdkit import Chem
from rdkit.Chem import MolFromSmiles, SDWriter, Draw
from rdkit.Chem import AllChem, AddHs
from rdkit.Chem import rdFingerprintGenerator
from rdkit.Chem.MolStandardize import rdMolStandardize
from tqdm import tqdm
import learn2learn as l2l

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

seed_everything(42)
torch.backends.cudnn.benchmark = True

def get_key_by_value(dictionary, value):
    for key, val in dictionary.items():
        if val == value:
            return key
    return None

In [ ]:
df = pd.read_excel('./henry.xlsx')
df.drop(['Unnamed: 0'], axis=1, inplace=True)

# keep only samples from df that have df['temps'] around 298
df = df[np.abs(df['temps'] - 298) < 5]
df = df.reset_index(drop=True)

# keep only tasks which have more than hiper_batch_size samples
df = df.groupby('smiles_solutes').filter(lambda x: len(x) > 4)
df = df.reset_index(drop=True)

df['Hcs'] = df['Hcs'] * 1000

# remove O=C=O samples
df = df[df['smiles_solutes'] != 'O=C=O']
df = df.reset_index(drop=True)

# load into df2 another excel file
df2 = pd.read_csv('carbon-dioxide.csv')
df2 = df2[df2['Split'] != 0]
df2 = df2.reset_index(drop=True)

df2['smiles_solutes'] = 'O=C=O'

pre_defined_split = False

test_indices = train_test_split(df2.index, test_size=0.2, random_state=0)[1]
df2.loc[test_indices, 'smiles_solutes'] = 'O=C=O_test'

df2.rename(columns={"Henry's law constant": 'Hcs', "Temperature": 'temps',
                    "SMILES": "smiles"}, inplace=True)
df2 = df2[['smiles', 'smiles_solutes', 'temps', 'Hcs']]

scaler = StandardScaler()
scaler.fit(df2['Hcs'].values.reshape(-1, 1))
df2['Hcs'] = scaler.transform(df2['Hcs'].values.reshape(-1, 1))
df['Hcs'] = scaler.transform(df['Hcs'].values.reshape(-1, 1))

# join df and df2
df = pd.concat([df, df2])
df = df.reset_index(drop=True)

# expand df['smiles'] based on dot into smiles_cation and smiles_anion
df['smiles_cation'] = ''
df['smiles_anion'] = ''
for i, row in df.iterrows():
    smiles = row['smiles']
    temp1, temp2 = smiles.split('.')
    if '+' in temp1:
        df.at[i, 'smiles_cation'] = temp1
        df.at[i, 'smiles_anion'] = temp2
    else:
        df.at[i, 'smiles_cation'] = temp2
        df.at[i, 'smiles_anion'] = temp1
df = df.reset_index(drop=True)

df

In [ ]:
df3 = pd.read_excel('./cosmo-predicted-hcs.xlsx')
df3['ils'] = df3['IL_cation'] + ' ' + df3['IL_anion']

# remove rows where henrycnodim == 0
df3 = df3[df3['henrycnodim'] != 0]
df3 = df3[df3['henryc'] != 0]
df3['henryc'] = df3['henryc'] * 101325 # units alignment
df3 = df3.reset_index(drop=True)

target_column_cosmo = 'henryc'
df3[target_column_cosmo] = 1 / (df3[target_column_cosmo])
df3['smiles'] = df3['smiles_cation'] + '.' + df3['smiles_anion']
df3['temps'] = 298
df3['smiles_solutes'] = df3['task'] + '_cosmo'
df3 = df3.dropna(subset=['smiles_anion'])
df3 = df3.reset_index(drop=True)
df3 = df3[['smiles', 'smiles_cation', 'smiles_anion', 'temps', target_column_cosmo, 'ils', 'smiles_solutes']]
df3.rename(columns={target_column_cosmo: 'Hcs'}, inplace=True)
q1 = df3['Hcs'].quantile(0.25)
q3 = df3['Hcs'].quantile(0.75)
iqr = q3 - q1
df3 = df3[(df3['Hcs'] >= q1 - 1.5 * iqr) & (df3['Hcs'] <= q3 + 1.5 * iqr)]
df3 = df3.reset_index(drop=True)

scaler3 = StandardScaler()
df3['Hcs'] = scaler3.fit_transform(df3['Hcs'].values.reshape(-1, 1))
df = pd.concat([df, df3])
df = df.reset_index(drop=True)
df

In [ ]:
nrm = rdMolStandardize.Normalizer()

def normalize_smiles(smile):
  cosmo_flag = False
  if '_cosmo' in smile:
    smile = smile.replace('_cosmo', '')
    cosmo_flag = True
  mol = Chem.MolFromSmiles(smile)
  mol_norm = nrm.normalize(mol)
  smile_norm = Chem.MolToSmiles(mol_norm, True)
  if cosmo_flag:
    smile_norm = smile_norm + '_cosmo'
  return smile_norm

smiles_cation_unique = df['smiles_cation'].unique()
smiles_anion_unique = df['smiles_anion'].unique()

smiles_cation_norm_dict = {}
smiles_anion_norm_dict = {}
for smile in smiles_cation_unique:
  smiles_cation_norm_dict[smile] = normalize_smiles(smile)
for smile in smiles_anion_unique:
  smiles_anion_norm_dict[smile] = normalize_smiles(smile)

df['smiles_cation'] = df['smiles_cation'].map(smiles_cation_norm_dict)
df['smiles_anion'] = df['smiles_anion'].map(smiles_anion_norm_dict)
df

In [ ]:
fold_to_use = 3

kf = KFold(n_splits=5, shuffle=True, random_state=0)
task = 'O=C=O'

# remove sufix '_valid' from samples in df
df['smiles_solutes'] = df['smiles_solutes'].str.replace('_valid', '')

# randomly select number of percentage of samples with this task
df_task = df[df['smiles_solutes'] == task]

for fold_index, (train_index, valid_index) in enumerate(kf.split(df_task)):
  if fold_index == fold_to_use:
    df.loc[df_task.iloc[valid_index].index, 'smiles_solutes'] = task + f'_valid'
    break  # Exit after marking the specified fold as validation

# Extract tasks
tasks = df['smiles_solutes'].values
tasks_dict = {task: i for i, task in enumerate(np.unique(tasks))}
tasks = np.array([tasks_dict[task] for task in tasks])

y = df['Hcs'].values

In [ ]:
def get_atom_features(smi):
  mol = Chem.MolFromSmiles(smi)
  mol = Chem.RemoveHs(mol)
  atomic_number = []
  num_hs = []
  hybr = []
  charges = []
  aromacity = []
  degrees = []

  for i, atom in enumerate(mol.GetAtoms()):
      atomic_number.append(atom.GetAtomicNum())
      num_hs.append(atom.GetTotalNumHs(includeNeighbors=True))
      hybr.append(atom.GetHybridization())

      charge = atom.GetFormalCharge()
      if np.isnan(charge):
          charge = 0
      charges.append(charge)

      aromacity.append(atom.GetIsAromatic())
      degrees.append(atom.GetDegree())

  le = LabelEncoder()
  hybr = le.fit_transform(hybr)

  result = np.array([atomic_number, num_hs, hybr, charges, aromacity, degrees])
  return np.transpose(result)

def get_edges_info(smi):
  mol = Chem.MolFromSmiles(smi)
  mol = Chem.RemoveHs(mol)
  row, col, bonds_types = [], [], []

  for i, bond in enumerate(mol.GetBonds()):
      start, end = bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()
      row += [start, end]
      col += [end, start]
      bonds_types += [bond.GetBondTypeAsDouble(), bond.GetBondTypeAsDouble()]
  return np.array([row, col], dtype=np.int_), np.array(bonds_types, dtype=np.float32)

memo = dict()

def dataframe_row_into_pytorch_geometric_molecular_graph(row, memo):
  if row['smiles'] in memo:
    x, edge_index, edge_attr = memo[row['smiles']]
  else:
    #calculate x for c and a
    c_x = get_atom_features(row['smiles_cation'])
    a_x = get_atom_features(row['smiles_anion'])

    temp_up = np.concatenate([c_x, np.zeros([c_x.shape[0],a_x.shape[1]])], axis=1)
    temp_down = np.concatenate([np.zeros([a_x.shape[0],c_x.shape[1]]), a_x], axis=1)
    b = np.concatenate([temp_up, temp_down]) #b for both c and a
    x = torch.tensor(b, dtype=torch.float)

    #calculate edges for c and a
    c_edge_index, c_edge_weights = get_edges_info(row['smiles_cation'])
    a_edge_index, a_edge_weights = get_edges_info(row['smiles_anion'])

    b = np.concatenate([c_edge_index, a_edge_index], axis=1)
    edge_index = torch.tensor(b, dtype=torch.long)

    b = np.concatenate([c_edge_weights, a_edge_weights])
    edge_attr = torch.tensor(b, dtype=torch.float)

    memo[row['smiles']] = (x, edge_index, edge_attr)
  y = torch.tensor(row['Hcs'], dtype=torch.float)
  id = row['ids']
  # ffv = torch.tensor(row['FFV'], dtype=torch.float)
  data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr,
              y=y, id=id, smiles=row['smiles'],
              # ffv=ffv
              )
  return data

In [ ]:
df['ids'] = tasks
graphs_list = []
for row in df.iterrows():
  row = row[1]
  data = dataframe_row_into_pytorch_geometric_molecular_graph(row, memo)
  graphs_list.append(data)
df['graphs'] = graphs_list
df

In [ ]:
def collate_fn(batch):
  ids = [data.id for data in batch]
  unique_ids = sorted(list(set(ids)))
  batches = []
  for id_ in unique_ids:
    same_id_data = [data for data in batch if data.id == id_]
    batches.append(Batch.from_data_list(same_id_data))
  return batches

# preapre loaders using this function
train_loader = DataLoader(df[~df['smiles_solutes'].isin(('O=C=O', 'O=C=O_valid', 'O=C=O_test'))]['graphs'].values, batch_size=32, shuffle=True, collate_fn=collate_fn)
adapt_loader = DataLoader(df[df['smiles_solutes'] == 'O=C=O']['graphs'].values, batch_size=32, shuffle=True)
valid_loader = DataLoader(df[df['smiles_solutes'] == 'O=C=O_valid']['graphs'].values, batch_size=32, shuffle=True)
test_loader = DataLoader(df[df['smiles_solutes'] == 'O=C=O_test']['graphs'].values, batch_size=32, shuffle=True)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self, conv_function, input_channels, embedding_size, linear_size, add_params_num=0):
        # Init parent
        super(GCN, self).__init__()
        self.crafted_add_params_num = add_params_num

        # GCN layers
        self.conv1 = conv_function(input_channels, embedding_size[0])
        self.conv2 = conv_function(embedding_size[0], embedding_size[1])
        self.conv3 = conv_function(embedding_size[1], embedding_size[2])
        self.conv4 = conv_function(embedding_size[2], embedding_size[3])

        # Dropout
        self.dropout1 = torch.nn.Dropout(0.2)

        # Linear layers
        self.linear1 = Linear(embedding_size[-1]+add_params_num, linear_size[0])
        self.linear2 = Linear(linear_size[0],linear_size[1])

        # Dropout 2
        self.dropout2 = torch.nn.Dropout(0.3)

        # batch normalization
        self.bnf = torch.nn.BatchNorm1d(linear_size[-1])

        # Output layer
        self.out = Linear(linear_size[-1], 1)


    def forward(self, x, edge_index, edge_weight, batch_index, cond=None):
        # Conv layers
        hidden = self.conv1(x, edge_index, edge_weight).relu()
        hidden = self.dropout1(hidden)
        hidden = self.conv2(hidden, edge_index, edge_weight).relu()
        hidden = self.dropout1(hidden)
        hidden = self.conv3(hidden, edge_index, edge_weight).relu()
        hidden = self.dropout1(hidden)
        hidden = self.conv4(hidden, edge_index, edge_weight).relu()
        hidden = self.dropout1(hidden)

        # Pooling
        hidden = gap(hidden, batch_index)

        # adding pressure and temperature info
        if self.crafted_add_params_num != 0:
            cond = cond.unsqueeze(1)
            hidden = torch.cat([hidden, cond], dim=1)

        # Apply a final (linear) classifier.
        hidden = self.linear1(hidden)
        hidden = self.linear2(hidden)
        hidden = self.dropout2(hidden)
        hidden = self.bnf(hidden)
        hidden = torch.nn.functional.relu(hidden)
        out = self.out(hidden)

        return out, hidden

In [ ]:
cond_names = [] #['FFV']

GCN_model = GCN(GCNConv, input_channels = df['graphs'].values[0].x.shape[1],
            embedding_size = [128,256,256,128], linear_size = [256,128],
            add_params_num = len(cond_names))

maml = l2l.algorithms.MAML(GCN_model, lr=1e-3).to(device)
optimizer_maml = torch.optim.Adam(maml.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer_maml, 40)
criterion = torch.nn.MSELoss()

In [ ]:
epochs = 30
train_losses = []
valid_losses = []
best_valid_loss = float('inf')

model = maml.clone().to(device)
for epoch in tqdm(range(epochs)):
    model.train()
    train_loss = 0
    r2 = 0
    for data in train_loader:

        model = maml.clone().to(device)

        data = data.to(device)
        optimizer_maml.zero_grad()
        if cond_names:
            out, _ = model(data.x, data.edge_index, data.edge_attr, data.batch, data.ffv)
        else:
            out, _ = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = criterion(out.squeeze(), data.y)

    model.adapt(loss)

    loss.backward()
    optimizer_maml.step()
    scheduler.step()
    train_loss += loss.detach().cpu().item()
    train_loss /= len(train_loader)

In [ ]:
model = maml.clone()
state_dict = model.module.state_dict()
model_to_adapt = GCN(GCNConv, input_channels = df['graphs'].values[0].x.shape[1],
            embedding_size = [128,256,256,128], linear_size = [256,128],
            add_params_num = len(cond_names))
model_to_adapt.load_state_dict(state_dict)
model_to_adapt = model_to_adapt.to(device)
optimizer = torch.optim.Adam(model_to_adapt.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, 40)
criterion = torch.nn.MSELoss()

epochs = 300
train_losses = []
valid_losses = []
r2s_valid = []
best_valid_loss = float('inf')

train_loss, valid_loss, r2 = 0, 0, 0

for epoch in (pbar := tqdm(range(epochs))):
    pbar.set_description(f"Loss: {train_loss:.3f}, valid_loss: {valid_loss:.3f}, R2: {r2:0.3f}")

    model_to_adapt.train()
    train_loss = 0
    r2 = 0
    for data in adapt_loader:
        data = data.to(device)
        optimizer.zero_grad()
        if cond_names:
            out, _ = model_to_adapt(data.x, data.edge_index, data.edge_attr, data.batch, data.ffv)
        else:
            out, _ = model_to_adapt(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = criterion(out.squeeze(), data.y)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.detach().cpu().item()
    train_loss /= len(train_loader)

    model_to_adapt.eval()
    valid_loss = 0
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in valid_loader:
            data = data.to(device)
            if cond_names:
                out, _ = model_to_adapt(data.x, data.edge_index, data.edge_attr, data.batch, data.ffv)
            else:
                out, _ = model_to_adapt(data.x, data.edge_index, data.edge_attr, data.batch)
            loss = criterion(out.squeeze(), data.y)
            valid_loss += loss.detach().cpu().item()
            y_true.extend(data.y.cpu().numpy())
            y_pred.extend(out.squeeze().cpu().numpy())
    valid_loss /= len(valid_loader)
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate metrics using permetrics
    metric = RegressionMetric(y_true, y_pred)
    rmse = metric.get_metric_by_name('RMSE')['RMSE']
    r2 = metric.get_metric_by_name('R2')['R2']
    mae = metric.get_metric_by_name('MAE')['MAE']
    mare = metric.get_metric_by_name('MAPE')['MAPE']

    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    r2s_valid.append(r2)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model_to_adapt.state_dict(), 'best_model.pth')
        best_train_loss = train_loss
        best_valid_loss = valid_loss
        best_r2 = r2
        best_rmse = rmse
        best_mae = mae
        best_mare = mare

print(f'\nTrain Loss: {best_train_loss:.4f} | Valid Loss: {best_valid_loss:.4f}', end=' | ')
print(f'RMSE: {best_rmse:.4f} | R2: {best_r2:.4f} | MAE: {best_mae:.4f} | MARE: {best_mare:.4f}')

In [ ]:
# plot train_losses vs epoch
plt.figure(figsize=(15,4))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(valid_losses, label='Valid Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# plot another plot with r2 vs epoch
plt.subplot(1, 2, 2)
plt.plot(r2s_valid, label='R2')
plt.xlabel('Epoch')
plt.ylabel('R2')
plt.legend()

plt.show()

In [ ]:
# Load the best model
model_to_adapt.load_state_dict(torch.load('best_model.pth'))

# Function to perform testing and print metrics
def test_model(loader, name):
    model_to_adapt.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            if cond_names:
                out, _ = model_to_adapt(data.x, data.edge_index, data.edge_attr, data.batch, data.ffv)
            else:
                out, _ = model_to_adapt(data.x, data.edge_index, data.edge_attr, data.batch)
            y_true.extend(data.y.cpu().numpy())
            y_pred.extend(out.squeeze().cpu().numpy())
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    metric = RegressionMetric(y_true, y_pred)
    rmse = metric.get_metric_by_name('RMSE')['RMSE']
    r2 = metric.get_metric_by_name('R2')['R2']
    mae = metric.get_metric_by_name('MAE')['MAE']
    mare = metric.get_metric_by_name('MAPE')['MAPE']

    # print(f"{name} Set Metrics: R2, RMSE, MAE, MARE")
    print(f"{r2:.4f}")
    print(f"{rmse:.4f}")
    print(f"{mae:.4f}")
    print(f"{mare:.4f}")
    print()

# Perform testing on train, validation, and test sets
print("Training - Validation - Test : R2, RMSE, MAE, MARE")
test_model(adapt_loader, "Training")
test_model(valid_loader, "Validation")
test_model(test_loader, "Test")